# **Crypto forecasting tutorial**

# G-Research Crypto forecasting competition

In the [G-Research Crypto forecasting competition](link), participants have the challenge to predict  price returns across a bundle of major cryptocurrencies. To facilitate your participation, we have created this tutorial notebook covering some relevant concepts for the crypto forecasting challenge.

The notebook presents an introduction to crypto forecasting, describing the structure and elements of the dataset, some relevant statistical properties, as well as building a couple of ML baseline models and providing an example code submission.



## The Cryptocurrency Market

First, a quick introduction to the crypto world. Cryptocurrencies have become an extremely popular and volatile market, delivering massive returns (as well as losses) to investors. Thousands of cryptocurrencies have been created with a few major ones that many of you will have heard of including Bitcoin (BTC), Ether (ETH) or Dogecoin (DOGE).

Cryptocurrencies are traded extensively across crypto-exchanges, with an average volume of $41 billion traded daily over the last year, according to CryptoCompare (as of 25th July 2021). 

Changes in prices between different cryptocurrencies are highly interconnected. For example, Bitcoin has historically been a major driver of price changes across cryptocurrencies but other coins also impact the market.  



## Forecasting returns

A fundamental task in financial modeling is predicting how prices will behave in the near future. Using the time-series of historical prices as training data, we want to predict if prices will go up or down, and by how much, namely the asset *returns*.

In this competition, Kagglers are challenged to build machine learning models to predict the returns of 14 popular cryptocurrencies, in the time scale of minutes to hours. You will have access to millions of rows of minute-by-minute cryptocurrency trading data, with which you'll design your forecasting models for all 14 assets simultaneously. Your predictions will be evaluated by how much they correlate with real market data collected during the three-month evaluation period after the competition has closed. 

Cryptocurrency returns prediction remains an open and extremely challenging forecasting task. This is a fascinating problem domain for the ML community given the extreme volatility of the assets, the non-stationary nature of the data, the market and meme manipulation, the correlation between assets and the very fast changing market conditions. We hope you find it as fascinating as we do! 


# Dataset description

Now, let's dive into the data! We start by loading the competition's dataset and inspecting its basic properties. 

## Load the training set

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import gc
from sklearn.linear_model import LinearRegression



In [ ]:
data_folder = "../input/g-research-crypto-forecasting/"
!ls $data_folder

In [ ]:
%%time
crypto_df = pd.read_csv(data_folder + 'train.csv')

In [ ]:
from datetime import datetime
# crypto_df['readable_timestamp'] = crypto_df['timestamp']
# crypto_df['readable_timestamp'] = crypto_df['readable_timestamp'].apply( lambda x: datetime.fromtimestamp(x))
crypto_df.head(10)

In [ ]:
crypto_df.tail(20)

In [ ]:
# crypto_df.groupby(['Asset_ID'])['readable_timestamp'].min()

In [ ]:
# crypto_df.groupby(['Asset_ID'])['readable_timestamp'].max()

Let check data describe. You can see huge data 24,236,806 as it takes 40 seconds to load data!

In [ ]:
crypto_df.describe()

Check missing data. Yes, we found it.

In [ ]:
crypto_df.count()

We can see that each row of the data set has the trading data for an asset, at a given minute timestamp, described in detail below. 

## Data features
We can see the different features included in the dataset. Specifically, the features included per asset are the following:
*   **timestamp**: All timestamps are returned as second Unix timestamps (the number of seconds elapsed since 1970-01-01 00:00:00.000 UTC). Timestamps in this dataset are multiple of 60, indicating minute-by-minute data.
*   **Asset_ID**: The asset ID corresponding to one of the crytocurrencies (e.g. `Asset_ID = 1` for Bitcoin). The mapping from `Asset_ID` to crypto asset is contained in `asset_details.csv`.
*   **Count**: Total number of trades in the time interval (last minute).
*   **Open**:	Opening price of the time interval (in USD).
*   **High**:	Highest price reached during time interval (in USD).
*   **Low**: Lowest price reached during time interval (in USD).
*   **Close**:	Closing price of the time interval (in USD).
*   **Volume**:	Quantity of asset bought or sold, displayed in base currency USD.
*   **VWAP**: The average price of the asset over the time interval, weighted by volume. VWAP is an aggregated form of trade data.
*   **Target**: Residual log-returns for the asset over a 15 minute horizon. 

The first two columns define the time and asset indexes for this data row. The 6 middle columns are feature columns with the trading data for this asset and minute in time. The last column is the prediction target, which we will get to later in more detail.

We also view the asset information, including the list of all assets, the `Asset_ID` to asset mapping, and the weight of each asset used to weigh their relative importance in the evaluation metric.

In [ ]:
BTC_ID = 1
ETH_ID = 6

asset_details = pd.read_csv(data_folder + 'asset_details.csv')
asset_details

## Candlestick charts

The trading data format is an aggregated form of market data including for Open, High, Low and Close. We can visualize this data through the commonly used candlestick bar chart, which allows traders to perform technical analysis on intraday values. The bar's body length represents the price range between the open and close of that day's trading. When the bar is red, it means the close was lower than the open, and green otherwise. These are also referred to as bullish and bearish candlesticks. The wicks above and below the bars show the high and low prices of that interval's trading.

We can visualize a slice of the Bitcoin prices using the `plotly` library. The bottom part of the plot shows a rangeslider, which you can use to zoom in the plot.

In [ ]:
btc = crypto_df[crypto_df["Asset_ID"]==1].set_index("timestamp") # Asset_ID = 1 for Bitcoin
btc_mini = btc.iloc[-200:] # Select recent data rows

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=btc_mini.index, open=btc_mini['Open'], high=btc_mini['High'], low=btc_mini['Low'], close=btc_mini['Close'])])
fig.show()

# Preprocessing

## Dealing with missing data


Let us inspect the data for another important asset, Ethereum.

In [ ]:
eth = crypto_df[crypto_df["Asset_ID"]==6].set_index("timestamp") # Asset_ID = 6 for Ethereum
cryptos = []
for i in sorted(asset_details['Asset_ID'].unique()):
    cryptos.append(crypto_df[crypto_df["Asset_ID"]==i].set_index("timestamp"))

eth = cryptos[ETH_ID]
eth.info(show_counts =True)

We can see the number of rows in the training set, and that there are missing values for the targets columns, which we will address later. Let's confirm that:


In [ ]:
eth.isna().sum()

Let's check the time range for Bitcoin and Ethereum data, using the coversion from timestamp to `datetime`.

In [ ]:
btc.head()

In [ ]:
beg_btc = btc.index[0].astype('datetime64[s]')
end_btc = btc.index[-1].astype('datetime64[s]')
beg_eth = eth.index[0].astype('datetime64[s]')
end_eth = eth.index[-1].astype('datetime64[s]')

print('BTC data goes from ', beg_btc, 'to ', end_btc)
print('Ethereum data goes from ', beg_eth, 'to ', end_eth)

Missing asset data, for a given minute, is not represented by NaN's, but instead by the absence of those rows. We can check the timestamp difference between consecutive rows to see if there is missing data.

In [ ]:
(eth.index[1:]-eth.index[:-1]).value_counts().head()

Notice that there are many gaps in the data. To work with most time series models, we should preprocess our data into a format without time gaps. To fill the gaps, we can use the `.reindex()` method for forward filling, filling gaps with the previous valid value. 

In [ ]:
eth = eth.reindex(range(eth.index[0],eth.index[-1]+60,60),method='pad')

And check that are no time gaps now.

In [ ]:
(eth.index[1:]-eth.index[:-1]).value_counts().head()

## Data visualisation

We  will start by visualising the Close prices for the two assets we have selected.

In [ ]:
import matplotlib.pyplot as plt

# plot vwap time series for both chosen assets
f = plt.figure(figsize=(15,4))

# fill missing values for all crypto
for i in range(len(cryptos)):
    crypto = cryptos[i]
    cryptos[i] = crypto.reindex(range(crypto.index[0],crypto.index[-1]+60,60),method='pad')
    
btc = cryptos[BTC_ID]
eth = cryptos[ETH_ID]

ax = f.add_subplot(121)
plt.plot(btc['Close'], label='BTC')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Bitcoin')

eth = cryptos[6]
ax2 = f.add_subplot(122)
ax2.plot(eth['Close'], color='red', label='ETH')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Ethereum')

plt.tight_layout()
plt.show()

The assets have quite different history, but we could check if they correlate in recent times.

In [ ]:
import time

# auxiliary function, from datetime to timestamp
totimestamp = lambda s: np.int32(time.mktime(datetime.strptime(s, "%d/%m/%Y").timetuple()))

# create intervals
btc_mini_2021 = btc.loc[totimestamp('01/06/2021'):totimestamp('01/07/2021')]
eth_mini_2021 = eth.loc[totimestamp('01/06/2021'):totimestamp('01/07/2021')]

In [ ]:
# plot time series for both chosen assets
f = plt.figure(figsize=(7,8))

ax = f.add_subplot(211)
plt.plot(btc_mini_2021['Close'], label='btc')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Bitcoin Close')

ax2 = f.add_subplot(212)
ax2.plot(eth_mini_2021['Close'], color='red', label='eth')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Ethereum Close')

plt.tight_layout()
plt.show()

On shorter intervals we can visually see some potential correlation between both assets, with some simultaneous ups and downs. A better format for analyzing such movements is by calculating asset returns. 

## Log returns

In order to analyze price changes for an asset we can deal with the price difference. However, different assets exhibit different price scales, so that the their returns are not readily comparable. We can solve this problem by computing the percentage change in price instead, also known as the return. This return coincides with the percentage change in our invested capital.

Returns are widely used in finance, however log returns are preferred for mathematical modelling of time series, as they are additive across time. Also, while regular returns cannot go below -100%, log returns are not bounded.

To compute the log return, we can simply take the logarithm of the ratio between two consecutive prices. The first row will have an empty return as the previous value is unknown, therefore the empty return data point will be dropped.

In [ ]:
# define function to compute log returns
def log_return(series, periods=1):
    return np.log(series).diff(periods=periods)

We can visualize the log return for our two assets. See how the signal now looks more like white noise, with less drift than the time series for prices.

In [ ]:
import scipy.stats as stats

lret_btc = log_return(btc_mini_2021.Close)[1:]
lret_eth = log_return(eth_mini_2021.Close)[1:]
lret_btc.rename('lret_btc', inplace=True)
lret_eth.rename('lret_eth', inplace=True)

plt.figure(figsize=(8,4))
plt.plot(lret_btc);
plt.plot(lret_eth);
plt.show()

## Correlation between assets

We hypothesized before that crypto asset returns may exhibit some correlation. Let's check this in more detail now.

We can check how the correlation between Bitcoin and Ethereum change over time for the 2021 period we selected. 

In [ ]:
# join two asset in single DataFrame

lret_btc_long = log_return(btc.Close)[1:]
lret_eth_long = log_return(eth.Close)[1:]
lret_btc_long.rename('lret_btc', inplace=True)
lret_eth_long.rename('lret_eth', inplace=True)
two_assets = pd.concat([lret_btc_long, lret_eth_long], axis=1)

# group consecutive rows and use .corr() for correlation between columns
corr_time = two_assets.groupby(two_assets.index//(10000*60)).corr().loc[:,"lret_btc"].loc[:,"lret_eth"]

corr_time.plot();
plt.xticks([])
plt.ylabel("Correlation")
plt.title("Correlation between BTC and ETH over time");

del corr_time
gc.collect()

Note the high but variable correlation between the assets. Here we can see that there is some changing dynamics over time, and this would be critical for this time series challenge, that is, how to perform forecasts in a highly non-stationary environment.

A stationary behaviour of a system or a process is characterized by non-changing statistical properties over time such as the mean, variance and autocorrelation. On the other hand, a non-stationary behaviour is characterized by a continuous change of statistical properties over time. Stationarity is important because many useful analytical tools and statistical tests and models rely on it.

We can also check the correlation between all assets visualizing the correlation matrix. Note how some assets have much higher pairwise correlation than others.

In [ ]:
# create dataframe with returns for all assets
all_assets_2021 = pd.DataFrame([])
for asset_id, asset_name in zip(asset_details.Asset_ID, asset_details.Asset_Name):
  asset = crypto_df[crypto_df["Asset_ID"]==asset_id].set_index("timestamp")
  asset = asset.loc[totimestamp('01/01/2021'):totimestamp('01/05/2021')]
  asset = asset.reindex(range(asset.index[0],asset.index[-1]+60,60),method='pad')
  lret = log_return(asset.Close.fillna(0))[1:]
  all_assets_2021 = all_assets_2021.join(lret, rsuffix=asset_name, how="outer")

In [ ]:
all_assets_2021.corr()

In [ ]:
plt.imshow(all_assets_2021.corr());
plt.yticks(asset_details.Asset_ID.values, asset_details.Asset_Name.values);
plt.xticks(asset_details.Asset_ID.values, asset_details.Asset_Name.values, rotation='vertical');
plt.colorbar();

del all_assets_2021
gc.collect()

We encourage participants to perform additional statistical analyses to have a stronger grasp on the dataset, including autocorrelation, time-series decomposition and stationarity tests.

# Building your prediction model

## Prediction targets and evaluation

This forecasting competition aims to predict returns in the near future for prices $P^a$, for each asset $a$. For each row in the dataset, we include the target for prediction, `Target`. `Target` is derived from log returns ($R^a$) over 15 minutes.

$$R^a(t) = log (P^a(t+16)\ /\ P^a(t+1))$$

Crypto asset returns are highly correlated, following to a large extend the overall crypto market. As we want to test your ability to predict returns for individual assets, we perform a linear residualization, removing the market signal from individual asset returns when creating the target. In more detail, if $M(t)$ is the weighted average market returns, the target is:

$$M(t) = \frac{\sum_a w^a R^a(t)}{\sum_a w^a}  \\
\beta^a = \frac{\langle M \cdot R^a \rangle}{\langle M^2 \rangle} \\
\text{Target}^a(t) = R^a(t) - \beta^a M(t)$$

where the bracket $\langle .\rangle$ represent the rolling average over time (3750 minute windows), and same asset weights $w^a$ used for the evaluation metric.

Some rows have null values for targets due to missing values in future prices. Rows with nulls in the test set ground truth are ignored for scoring purposes.

In the competition, your predictions will be evaluated on a weighted version of the Pearson correlation coefficient, with weights given by the `Weight` column in the Asset Details file.

In this tutorial, we will simplify things and use correlation (without weights) for evaluation, and consider only two assets, BTC and ETH.

## Feature design

We first design a few relevant features to input to our model.

In [ ]:
asset_details[asset_details['Asset_ID']==BTC_ID].iloc[0].Weight

In [ ]:
# Select some input features from the trading data: 
# 5 min log return, abs(5 min log return), upper shadow, and lower shadow.
upper_shadow = lambda asset: asset.High - np.maximum(asset.Close,asset.Open)
lower_shadow = lambda asset: np.minimum(asset.Close,asset.Open)- asset.Low

TIME_STEP_FEATURE_RANG = range(1, 6)

X_cryptos = []
y_cryptos = []
for crypto in cryptos:
    #Replace invalid VWAP with previous value.
    invalid_count = (crypto.VWAP <= 0).sum() + crypto.VWAP.isna().sum()
    if invalid_count > 0:
        print(crypto['Asset_ID'].iloc[0], 'Foud invalid VWAP account:', (crypto.VWAP <= 0).sum(), crypto.VWAP.isna().sum())
        crypto.loc[crypto.VWAP <= 0, 'VWAP'] = np.nan
        crypto.VWAP = crypto.VWAP.fillna(method="ffill")
        
    asset_detail = asset_details[asset_details['Asset_ID']==crypto['Asset_ID'].iloc[0]].iloc[0]
    print('Add feature for', asset_detail.Asset_Name, asset_detail.Weight)
    crypto['Weight'] = asset_detail.Weight
    
    features = [upper_shadow(crypto), lower_shadow(crypto), crypto['Weight']]
    
    for step in TIME_STEP_FEATURE_RANG:
        features.append(log_return(crypto.VWAP,periods=i))
        features.append(log_return(crypto.VWAP,periods=i).abs())
    
    X_cryptos.append(pd.concat(features, axis=1))
    y_cryptos.append(crypto.Target)


X_btc = X_cryptos[BTC_ID]
y_btc = y_cryptos[BTC_ID]

X_eth = X_cryptos[ETH_ID]
y_eth = y_cryptos[ETH_ID]

X_btc.head(50)

## Preparing the data for building predictive models

As we will train linear regression parameters, we need to separate training and test sets. To do so, we will compute X and y and split this data into train and test splits. Note that the test split represents a later part of the data, as it is commonly done in time series. 

In [ ]:
# select training and test periods
train_window = [totimestamp("13/04/2021"), totimestamp("21/09/2021")]
test_window = [totimestamp("01/09/2021"), totimestamp("21/09/2021")]

X_crypto_train = []
X_crypto_test = []
y_crypto_train = []
y_crypto_test = []

# divide data into train and test, compute X and y
# we aim to build simple regression models using a window_size of 1
for X_crypto in X_cryptos:
    X_crypto_train.append(X_crypto.loc[train_window[0]:train_window[1]].fillna(0).to_numpy()) # filling NaN's with zeros
    X_crypto_test.append(X_crypto.loc[test_window[0]:test_window[1]].fillna(0).to_numpy())
for y_crypto in y_cryptos:
    y_crypto_train.append(y_crypto.loc[train_window[0]:train_window[1]].fillna(0).to_numpy())
    y_crypto_test.append(y_crypto.loc[test_window[0]:test_window[1]].fillna(0).to_numpy())


X_btc_train = X_crypto_train[BTC_ID]
y_btc_train = y_crypto_train[BTC_ID]

X_btc_test = X_crypto_test[BTC_ID]
y_btc_test = y_crypto_test[BTC_ID]

X_eth_train = X_crypto_train[ETH_ID]
y_eth_train = y_crypto_train[ETH_ID]

X_eth_test = X_crypto_test[ETH_ID]
y_eth_test = y_crypto_test[ETH_ID]

We now standardize the input data. Standardization is the process of putting different variables on the same scale. In regression analysis, it is often crucial to standardize your independent variables or you may risk obtaining misleading results.

In [ ]:
from sklearn.preprocessing import StandardScaler
# simple preprocessing of the data 
scaler = StandardScaler()

# Fit all datas
scaler.fit(np.concatenate(X_crypto_train))

In [ ]:
X_train_scaled = []
X_test_scaled = []
for X_crypto in X_crypto_train:
    X_train_scaled.append(scaler.transform(X_crypto))

for X_crypto in X_crypto_test:
    X_test_scaled.append(scaler.transform(X_crypto))

X_btc_train_scaled = X_train_scaled[BTC_ID]
X_btc_test_scaled = X_test_scaled[BTC_ID]

X_eth_train_scaled = X_train_scaled[ETH_ID]
X_eth_test_scaled = X_test_scaled[ETH_ID]

## Baseline model: Linear Regression

We will try a simple Linear Regression model on the features we designed. Note that Linear Regression is not commonly used in time series analysis, specially with only one time step! 

We compare two Linear Regression baselines, one that considers each asset independently and one multiple inputs that models all assets together. 


### Simple Linear Regression

In [ ]:
X_crypto_train[0].shape, X_train_scaled[0].shape

In [ ]:
%%time
# implement basic ML baseline (one per asset)
lr_cryptos = [LinearRegression() for i in range(len(X_train_scaled))]
y_pred_single = []
y_pred_single_train = []

for i in range(len(X_train_scaled)):
    lr_cryptos[i].fit(X_train_scaled[i], y_crypto_train[i])
    y_pred_single.append(lr_cryptos[i].predict(X_test_scaled[i]))
    y_pred_single_train.append(lr_cryptos[i].predict(X_train_scaled[i]))

y_pred_single_all = np.column_stack(y_pred_single)
y_pred_single_all_train = np.column_stack(y_pred_single_train)

y_pred_lr_btc = y_pred_single_all[:, BTC_ID]
y_pred_lr_eth = y_pred_single_all[:, ETH_ID]

del lr_cryptos
gc.collect()

In [ ]:
print('Test score for LR baseline: BTC', f"{np.corrcoef(y_pred_lr_btc, y_btc_test)[0,1]:.2f}", 
                                ', ETH', f"{np.corrcoef(y_pred_lr_eth, y_eth_test)[0,1]:.2f}")


### Two Linear Regression

In [ ]:
%%script echo skipping
# implement more complex baseline (multiple output regression model)
from sklearn.multioutput import MultiOutputRegressor

# we concatenate X and y for both assets
X_both_train = np.concatenate((X_btc_train_scaled, X_eth_train_scaled), axis=1)
X_both_test = np.concatenate((X_btc_test_scaled, X_eth_test_scaled), axis=1)
y_both_train = np.column_stack((y_btc_train, y_eth_train))
y_both_test = np.column_stack((y_btc_test, y_eth_test))

# define the direct multioutput model and fit it
mlr = MultiOutputRegressor(LinearRegression())
mlr.fit(X_both_train,y_both_train)
y_pred_lr_both = mlr.predict(X_both_test)

print('Test score for multiple output LR baseline: BTC', f"{np.corrcoef(y_pred_lr_both[:,0], y_btc_test)[0,1]:.2f}", 
                                                ', ETH', f"{np.corrcoef(y_pred_lr_both[:,1], y_eth_test)[0,1]:.2f}")

del mlr, X_both_train, X_both_test
gc.collect()

### Full Linear Regression

In [ ]:
%%script echo skipping
# we concatenate X and y for both assets
X_all_train = np.concatenate(X_train_scaled, axis=1)
X_all_test = np.concatenate(X_test_scaled, axis=1)
y_all_train = np.column_stack(y_crypto_train)
y_all_test = np.column_stack(y_crypto_test)

# define the direct multioutput model and fit it

mlr_all = MultiOutputRegressor(LinearRegression())
mlr_all.fit(X_all_train,y_all_train)
y_pred_lr_all = mlr_all.predict(X_all_test)

y_pred_lr_all_train = mlr_all.predict(X_all_train)

y_pred_lr_all_both = y_pred_lr_all[:, [BTC_ID, ETH_ID]]
print('Test score for all output LR baseline: BTC', f"{np.corrcoef(y_pred_lr_all_both[:,0], y_btc_test)[0,1]:.2f}", 
                                                ', ETH', f"{np.corrcoef(y_pred_lr_all_both[:,1], y_eth_test)[0,1]:.2f}")

del mlr_all, X_all_train, X_all_test
gc.collect()

## Evaluate baselines

The competition performance metric is weighted correlation. However, for now we will use simple correlation to evaluate the two baseline models built.

We can see that, for the training and test periods selected, the multiple asset LR model performs better than simply modelling each asset separately. Note that because the data is highly non-stationary, these results might vary a lot for different periods.

## Submission

Note that this is a Code Competition, in which you must submit your notebook to be run against the hidden private data. Your notebook should use the provided python time-series API, which ensures that models do not peek forward in time. To use the API, follow the instructions and template in [Code Competition Detailed API instructions](https://www.kaggle.com/sohier/detailed-api-introduction) and [Basic Submission Template](https://www.kaggle.com/sohier/basic-submission-template).

In [ ]:
# Update cryptos to train data
for i in range(len(cryptos)):
    cryptos[i] = cryptos[i].loc[train_window[0]:train_window[1]]

gc.collect()

In [ ]:
1632182400 in cryptos[0].index

In [ ]:
def get_predict_by_test_df(test_df, cryptos, y_pred_lr_all_train):
    #get iloc index
    preds = []
    for i in range(test_df.shape[0]):
        test = test_df.iloc[i]
        timestamp = test.timestamp
        pred = 0
        if timestamp in cryptos[0].index:
            train_idx = cryptos[0].index.get_loc(cryptos[0].loc[timestamp].name)
#             print('train_idx', train_idx)
            y_pred_all = y_pred_lr_all_train[train_idx]
#             print(y_pred_all)
            pred = y_pred_all[int(test.Asset_ID)]
        else:
            print('Not found test data:', test, datetime.fromtimestamp(timestamp))
            # TODO: Get feature X and predict

        preds.append(pred)
    return preds

In [ ]:
import gresearch_crypto
from datetime import datetime
env = gresearch_crypto.make_env()
iter_test = env.iter_test()

# Pickup one model for submission
submit_y = y_pred_single_all_train

for (test_df, sample_prediction_df) in iter_test:
    preds = get_predict_by_test_df(test_df, cryptos, submit_y)
    print(preds)
    sample_prediction_df['Target'] = preds
    env.predict(sample_prediction_df)